In [ ]:
from preprocess import get_data

import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
data = get_data()

In [3]:
X = data["values"]
X

array([[ -56.,  -50.,  -64., ...,  -84.,  -91.,  -96.],
       [ -91.,  -78.,  -75., ...,  -17.,  -27.,  -42.],
       [ -61.,  -59.,  -46., ..., -120., -111., -118.],
       ...,
       [ 411.,  422.,  401., ...,  136.,  102.,   87.],
       [  53.,   -8.,  -55., ...,  293.,  239.,  180.],
       [  93.,   24.,  -51., ..., -231., -272., -272.]], dtype=float32)

In [4]:
def rename_labels(labels: np.array) -> np.array:
    transformed_labels = np.zeros_like(labels)
    
    # Apply the transformation logic
    transformed_labels[labels == 1] = 2  # seizure
    transformed_labels[(labels == 2) | (labels == 3)] = 1  # risk
    return transformed_labels # other without risk

Y = rename_labels(data["labels"])
Y

array([0, 0, 0, ..., 2, 2, 2], dtype=int32)

In [5]:
# Split for train and test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

# Scale and normalize data
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled =  scaler.transform(X_test)

In [6]:
svm_model = SVC(kernel='rbf', C=69)
svm_model.fit(X_train_scaled, y_train)

y_pred = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8999122036874452

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    # 'kernel': ['rbf', 'poly', 'linear']
    # 'kernel': ['rbf']
    'kernel': ['linear']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train_scaled, y_train)

best_model = grid.best_estimator_
print(grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   8.4s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   7.9s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   8.1s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   8.3s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   7.9s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   8.2s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   7.7s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   7.7s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   7.6s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   7.7s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   7.8s
[CV] END ...................C=0.1, gamma=0.01, k

In [10]:
svm_model = SVC(kernel='rbf', C=100, gamma=0.01)
svm_model.fit(X_train_scaled, y_train)

y_pred = svm_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.897278314310799

In [ ]:
def load_model():
    with open('svm_model.pkl', 'rb') as file:
        return pickle.load(file)

In [ ]:
def save_model():
    with open('svm_model.pkl', 'wb') as file:
        pickle.dump(svm_model, file)

In [ ]:
def predict(eeg_data: np.array) -> int:
    return svm_model.predict(eeg_data)